In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import deepchem as dc
import json
import numpy as np
from torch.utils.data import Dataset, DataLoader
import glob
import wandb
import os
import torch.optim as optimizers

2021-08-25 16:08:49.878644: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/cuda/extras/CUPTI/lib64/:/opt/intel/lib:/opt/intel/mkl/lib/intel64:/opt/intel:/opt/ibm/ILOG/CPLEX_Studio1210/cplex/bin/x86-64_linux:/opt/ibm/ILOG/CPLEX_Studio1210/cplex/python/3.7/x86-64_linux:/opt/intel/clck_latest/lib:/opt/intel/daal/lib:/opt/intel/intelpython3/lib:/opt/intel/ipp/lib:/opt/intel/itac_2019/lib:/opt/intel/itac_latest/lib:/opt/intel/mkl/lib:/opt/intel/mkl_/lib:/opt/intel/mpirt/lib:/opt/intel/tbb/lib:/opt/intel/clck/2019.0/lib:/opt/intel/compilers_and_libraries_2019/linux/lib:/opt/intel/compilers_and_libraries/linux/lib:/opt/intel/itac/2019.0.018/lib:/opt/intel/itac_2019/intel64/lib:/opt/intel/itac_latest/intel64/lib:/opt/intel/parallel_studio_xe_2019.0.045/clck_2019/lib:/opt/intel/parallel_studio_xe_2019.0.045/itac_2019/l

In [3]:
import dfs_code
from torch_geometric.data import InMemoryDataset, Data
import pickle
import torch
import torch.nn as nn
import random
import tqdm
import copy
#torch.multiprocessing.set_sharing_strategy('file_system') # this is important
# ulimit -n 500000
#def set_worker_sharing_strategy(worker_id: int) -> None:
#    torch.multiprocessing.set_sharing_strategy('file_system')

In [4]:
import sys
sys.path = ['/home/chrisw/Documents/projects/2021/graph-transformer/src'] + sys.path
from dfs_transformer import EarlyStopping, DFSCodeSeq2SeqFC, Deepchem2TorchGeometric, collate_minc_rndc_y, DFSCodeSeq2SeqFCLegacy

In [5]:
#wandb.init(project='moleculenet', entity='chrisxx')

config = wandb.config
config.n_atoms = 118
config.n_bonds = 4
config.emb_dim = 120
config.nhead = 12
config.nlayers = 6
config.max_nodes = 400
config.max_edges = 600
config.max_nodes_data = 400
config.max_edges_data = 600
config.onlyRandom = True
config.use_min = False # interestingly random seems to work better than using the minimal codes...
config.dim_feedforward = 2048
config.lr = 0.00003 # 10x smaller than the learning rate of the pretraining seems to be good
config.clip_gradient = 0.5
config.n_epochs = 25
config.patience = 3
config.factor = 0.95
config.minimal_lr = 6e-8
config.target_idx = 7
config.batch_size = 64
config.valid_patience = 5
config.valid_minimal_improvement = 0.00
config.pretrained_dir = '../../models/chembl/better_transformer/medium/'
config.model_dir = '../../models/moleculenet/tox21/medium2/'
config.num_workers = 0
config.load_last = True
config.dataset = 'clintox' # supported 'bbbp', 'clintox', 'hiv', 'tox21'
config.seed = 123

In [6]:
random.seed(config.seed)
torch.manual_seed(config.seed)
np.random.seed(config.seed)

In [7]:
os.makedirs(config.model_dir, exist_ok=True)

In [8]:
torch.multiprocessing.get_all_sharing_strategies()

{'file_descriptor', 'file_system'}

In [9]:
ngpu=1
device = torch.device('cuda:0' if (torch.cuda.is_available() and ngpu > 0) else 'cpu')

In [10]:
to_cuda = lambda T: [t.cuda() for t in T]

In [11]:
model = DFSCodeSeq2SeqFC(n_atoms=config.n_atoms,
                         n_bonds=config.n_bonds, 
                         emb_dim=config.emb_dim, 
                         nhead=config.nhead, 
                         nlayers=config.nlayers, 
                         max_nodes=config.max_nodes, 
                         max_edges=config.max_edges,
                         atom_encoder=nn.Embedding(config.n_atoms, config.emb_dim), 
                         bond_encoder=nn.Linear(config.n_bonds, config.emb_dim))

In [12]:
if config.load_last:
    model.load_state_dict(torch.load(config.pretrained_dir+'checkpoint.pt'))

In [36]:
def get_random_scaffold_split(dataset):
    splitter = dc.splits.ScaffoldSplitter()
    shuffled = dataset.complete_shuffle()
    trainidx, valididx, testidx = splitter.split(shuffled)
    train = shuffled.select(trainidx)
    valid = shuffled.select(valididx)
    test = shuffled.select(testidx)
    return train, valid, test

In [79]:
if config.dataset == 'clintox':
    tasks, datasets, transformers = dc.molnet.load_clintox(reload=False, featurizer=dc.feat.RawFeaturizer(True))
elif config.dataset == 'tox21':
    tasks, datasets, transformers = dc.molnet.load_tox21(reload=False, featurizer=dc.feat.RawFeaturizer(True))
elif config.dataset == 'hiv':
    tasks, datasets, transformers = dc.molnet.load_hiv(reload=False, featurizer=dc.feat.RawFeaturizer(True))
elif config.dataset == 'bbbp':
    tasks, datasets, transformers = dc.molnet.load_bbbp(reload=False, featurizer=dc.feat.RawFeaturizer(True))

Failed to featurize datapoint 7, None. Appending empty array
Exception message: Python argument types in
    rdkit.Chem.rdmolfiles.CanonicalRankAtoms(NoneType)
did not match C++ signature:
    CanonicalRankAtoms(RDKit::ROMol mol, bool breakTies=True, bool includeChirality=True, bool includeIsotopes=True)
Failed to featurize datapoint 302, None. Appending empty array
Exception message: Python argument types in
    rdkit.Chem.rdmolfiles.CanonicalRankAtoms(NoneType)
did not match C++ signature:
    CanonicalRankAtoms(RDKit::ROMol mol, bool breakTies=True, bool includeChirality=True, bool includeIsotopes=True)
Failed to featurize datapoint 983, None. Appending empty array
Exception message: Python argument types in
    rdkit.Chem.rdmolfiles.CanonicalRankAtoms(NoneType)
did not match C++ signature:
    CanonicalRankAtoms(RDKit::ROMol mol, bool breakTies=True, bool includeChirality=True, bool includeIsotopes=True)
Failed to featurize datapoint 984, None. Appending empty array
Exception messa

In [14]:
model_head = nn.Linear(5*config.emb_dim, 1)
model_head.to(device)
model.to(device)
print()

In [15]:
if config.dataset == 'tox21' or config.dataset == 'hiv' and not config.onlyRandom:
    if config.dataset == 'tox21':
        datadir = '../../datasets/moleculenet/tox21/srp53/'
    if config.dataset == 'hiv':
        datadir = '../../datasets/moleculenet/hiv_%d_%d/'%(config.max_nodes_data, config.max_edges_data)
    collate_fn = collate_minc_rndc_y
    trainloader = DataLoader(torch.load(datadir+'trainingset.pt'), batch_size=config.batch_size, shuffle=True, pin_memory=False, collate_fn=collate_fn)
    validloader = DataLoader(torch.load(datadir+'validationset.pt'), batch_size=config.batch_size, shuffle=False, pin_memory=False, collate_fn=collate_fn)
    testloader = DataLoader(torch.load(datadir+'testset.pt'), batch_size=config.batch_size, shuffle=False, pin_memory=False, collate_fn=collate_fn)
else:
    trainset, validset, testset = datasets
    collate_fn = collate_minc_rndc_y
    traindata = Deepchem2TorchGeometric(trainset, -1, onlyRandom=config.onlyRandom, max_nodes=config.max_nodes_data, max_edges=config.max_edges_data)
    validdata = Deepchem2TorchGeometric(validset, -1, onlyRandom=config.onlyRandom, max_nodes=config.max_nodes_data, max_edges=config.max_edges_data)
    testdata = Deepchem2TorchGeometric(testset, -1, onlyRandom=config.onlyRandom, max_nodes=config.max_nodes_data, max_edges=config.max_edges_data)
    trainloader = DataLoader(traindata, batch_size=config.batch_size, shuffle=True, pin_memory=False, collate_fn=collate_fn, num_workers=config.num_workers)
    validloader = DataLoader(validdata, batch_size=config.batch_size, shuffle=False, pin_memory=False, collate_fn=collate_fn, num_workers=config.num_workers)
    testloader = DataLoader(testdata, batch_size=config.batch_size, shuffle=False, pin_memory=False, collate_fn=collate_fn, num_workers=config.num_workers)

In [16]:
optim = optimizers.Adam(list(model.parameters()) + list(model_head.parameters()), lr=config.lr)

lr_scheduler = optimizers.lr_scheduler.ReduceLROnPlateau(optim, mode='min', verbose=True, patience=config.patience, factor=config.factor)
early_stopping_model = EarlyStopping(patience=config.valid_patience, delta=config.valid_minimal_improvement,
                              path=config.model_dir+'checkpoint_model.pt')
early_stopping_head = EarlyStopping(patience=config.valid_patience, delta=config.valid_minimal_improvement,
                              path=config.model_dir+'checkpoint_head.pt')
bce = torch.nn.BCEWithLogitsLoss()
sigmoid = nn.Sigmoid()

In [17]:
from sklearn.metrics import roc_auc_score, average_precision_score

def score(loader, model, model_head, use_min=False):
    val_roc = 0
    with torch.no_grad():
        full_preds, target = [], []
        for batch in tqdm.tqdm(loader):
            rndc, minc, z, eattr, y = batch
            if use_min:
                code = to_cuda(minc)
            else:
                code = to_cuda(rndc)
            y = y.to(device)
            self_attn, features, eos = model.encode(code, to_cuda(z), to_cuda(eattr)) # not clear whether to use minc or randc
            pred = sigmoid(model_head(features)).squeeze()
            
            pred_np = pred.detach().cpu().numpy().tolist()
            y_np = y.detach().cpu().numpy().tolist()
            full_preds += pred_np
            target += y_np

        target = np.asarray(target)
        full_preds = np.asarray(full_preds)
        roc = roc_auc_score(target, full_preds)
        prc = average_precision_score(target, full_preds)
    return roc, prc

In [18]:
valid_scores = []
try:
    for epoch in range(config.n_epochs):  
        epoch_loss = 0
        pbar = tqdm.tqdm(trainloader)
        model.train()
        for i, data in enumerate(pbar):
            optim.zero_grad()
            rndc, minc, z, eattr, y = data
            if config.use_min:
                code = to_cuda(minc)
            else:
                code = to_cuda(rndc)
            z = to_cuda(z)
            eattr = to_cuda(eattr)
            y = y.to(device)
            #prediction
            self_attn, features, eos = model.encode(code, z, eattr)
            prediction = model_head(features).squeeze()
            loss = bce(prediction, y)
            
            loss.backward()
            if config.clip_gradient is not None:
                torch.nn.utils.clip_grad_norm_(model.parameters(), config.clip_gradient)
            optim.step()
            epoch_loss = (epoch_loss*i + loss.item())/(i+1)
            
            wandb.log({'loss':epoch_loss})
            pbar.set_description('Epoch %d: CE %2.6f'%(epoch+1, epoch_loss))
        model.eval()
        roc_auc_valid, prc_auc_valid = score(validloader, model, model_head, config.use_min) 
        valid_scores += [roc_auc_valid]
        lr_scheduler.step(epoch_loss)
        early_stopping_model(-roc_auc_valid, model)
        early_stopping_head(-roc_auc_valid, model_head)
        curr_lr = list(optim.param_groups)[0]['lr']
        wandb.log({'roc_valid':roc_auc_valid, 'prc_valid':prc_auc_valid, 'learning rate':curr_lr})
        print('ROCAUC',roc_auc_valid, 'PRCAUC', prc_auc_valid)

        if early_stopping_model.early_stop:
            break

        if curr_lr < config.minimal_lr:
            break

except KeyboardInterrupt:
    torch.save(model.state_dict(), config.model_dir+'_keyboardinterrupt.pt')
    print('keyboard interrupt caught')


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  9.65it/s]


ROCAUC 0.6393110435663627 PRCAUC 0.14583803340456525


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  9.81it/s]


ROCAUC 0.6585612968591692 PRCAUC 0.09854405083126681


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  8.52it/s]


...

EarlyStopping counter: 3 out of 5
EarlyStopping counter: 3 out of 5
ROCAUC 0.6028368794326241 PRCAUC 0.13114047533061618


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  9.18it/s]


EarlyStopping counter: 4 out of 5
EarlyStopping counter: 4 out of 5
ROCAUC 0.5390070921985816 PRCAUC 0.12242252646349415


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  9.81it/s]

EarlyStopping counter: 5 out of 5
EarlyStopping counter: 5 out of 5
ROCAUC 0.5724417426545085 PRCAUC 0.12471151601451363


In [19]:
model.load_state_dict(torch.load(config.model_dir+'checkpoint_model.pt'))
model_head.load_state_dict(torch.load(config.model_dir+'checkpoint_head.pt'))
model.eval()
roc_auc_valid, prc_auc_valid = score(testloader, model, model_head) 
wandb.log({'roc_test':roc_auc_valid, 'prc_test':prc_auc_valid})

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 12.24it/s]


In [20]:
roc, prc = score(testloader, model, model_head, True)
wandb.log({'roc_test_min':roc, 'prc_test_min':prc})
print('ROC, PRC VALID', score(validloader, model, model_head, True))
print('ROC, PRC TEST', score(testloader, model, model_head, True))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  9.61it/s]


ROC, PRC VALID (0.6919959473150963, 0.09447251417062555)


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 12.76it/s]

ROC, PRC TEST (0.6557971014492754, 0.12283654698617698)


In [21]:
avg_roc = 0
avg_prc = 0
for i in range(20):
    roc, prc = score(testloader, model, model_head, False)
    avg_roc += roc
    avg_prc += prc
avg_roc /= 20
avg_prc /= 20 
wandb.log({'roc_test_avg20':avg_roc, 'prc_test_avg20':avg_prc})
print('ROC, PRC TEST', avg_roc, avg_prc)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  9.06it/s]

ROC, PRC TEST 0.665036231884058 0.12814474881844923
